# Lab 5 - Parcel Feature Extraction

Next, we will illustrate the construction of features related to our main task: finding the relationship between property development and water quality over time.  In a previous lab, you identified lakes for which we have complete information for the years from 2004 to 2015.  In this lab, we will

[Original Data and variable information](https://gisdata.mn.gov/organization/us-mn-state-metrogis?q=Metro+Regional+Parcel+Dataset&sort=score+desc%2C+metadata_modified+desc)

## Problem 1 - Feature construction

**Overview.** Remember that our target output file will have one row per year-lake combination.  To attach property information, we will need to group and aggregate the parcel data to create features for each lake-year combination.  When grouping the data, be sure to maintain the variables needed to join to the water quality data, namely the lake ID and year.  Since we are looking at tracking property development/change over time, we will want to generate features tracking

* Number of properties close to each lake,
* Summaries of the value of properties close to each lake,
* Aggregations on the size and type of the properties, and
* Other features that might impact water quality.
    
#### Task 1. Understanding parcel variables

Before we can construct features, we need to make sure we understand the parcel data.  The metro parcel data is provided by the State of Minnesota and the meta data can be found online.  For example, searching for *metro parcel 2014* lead to [this site](https://geo.btaa.org/catalog/304cf3d8-a53b-4ea9-b02a-f550bd68e320).  Clicking on the *Meta data* button in the top left, brought up more information.  Clicking *Download* opened in this meta data [in a separate page](https://resources.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_metrogis/plan_regonal_parcels_2014/metadata/metadata.html)

Look through the **Section 4: Attributes** and identify variables that might impact the water quality of near-by lakes.

> <font color="orange"> Potential Important Variables:
- USE(1-4)_DESC : property use types
- EMV_LAND, EMV_BLDG, EMV_TOTAL : est market value of land, buildings, total
- ACRES_POLY, ACRES_DEED : acreage of the land
- BASEMENT : basement (Y/N)
- GARAGE, GARAGESQFT : large concrete surface? garage is Y/N
- FIN_SQ_FT: sq footage
- YEAR_BUILT
- NUM_UNITS
- GREEN_ACRE, OPEN_SPACE, AG_PRESERV : undeveloped land / open areas
- TOTAL_TAX, TAX_CAPAC
- SPEC_ASSES : special assessments </font>

In [10]:
import polars as pl
import polars.selectors as cs

In [23]:
(wq := pl.read_parquet('./data/water_quality_by_year.parquet'))

DNR_ID_Site_Number,LAKE_NAME,Year,latitude,longitude,avg_secchi_depth,avg_total_phosphorus
str,str,i64,str,str,f64,f64
"""82007700-01""","""Goggins Lake""",2006,"""45.13303508""","""-92.89283249""",0.956,0.102
"""19002700-01""","""Crystal Lake""",2014,"""44.72296805""","""-93.27036573""",1.96,0.0244
"""82012200-01""","""Pine Tree Lake""",2004,"""45.10231359""","""-92.95386928""",2.2625,0.027375
"""10001900-01""","""Bavaria Lake""",2013,"""44.83812233""","""-93.63778927""",1.2,0.034636
"""27062700-01""","""Northwood Lake""",2010,"""45.02556284""","""-93.39171496""",0.98,0.1369
…,…,…,…,…,…,…
"""27003501-01""","""Sweeney Lake""",2006,"""44.99052075""","""-93.34160616""",1.03,0.0939
"""82009700-01""","""La Lake""",2006,"""44.88725237""","""-92.9713984""",1.475,0.096333
"""82011602-01""","""Armstrong Lake""",2008,"""44.96252306""","""-92.93917709""",1.142857,0.054714


In [39]:
# Load parcel data using lazy evaluation
(parcels_lazy := pl.scan_delta('./data/parcel.delta') )

In [17]:
parcels_lazy.collect_schema().names()

['ACRES_DEED',
 'ACRES_POLY',
 'AGPRE_ENRD',
 'AGPRE_EXPD',
 'AG_PRESERV',
 'BASEMENT',
 'BLDG_NUM',
 'BLOCK',
 'CITY',
 'CITY_USPS',
 'COOLING',
 'COUNTY_ID',
 'DWELL_TYPE',
 'EMV_BLDG',
 'EMV_LAND',
 'EMV_TOTAL',
 'FIN_SQ_FT',
 'GARAGE',
 'GARAGESQFT',
 'GREEN_ACRE',
 'HEATING',
 'HOMESTEAD',
 'HOME_STYLE',
 'LANDMARK',
 'LOT',
 'MULTI_USES',
 'NUM_UNITS',
 'OPEN_SPACE',
 'OWNER_MORE',
 'OWNER_NAME',
 'OWN_ADD_L1',
 'OWN_ADD_L2',
 'OWN_ADD_L3',
 'PARC_CODE',
 'PIN',
 'PLAT_NAME',
 'PREFIXTYPE',
 'PREFIX_DIR',
 'SALE_DATE',
 'SALE_VALUE',
 'SCHOOL_DST',
 'SPEC_ASSES',
 'STREETNAME',
 'STREETTYPE',
 'SUFFIX_DIR',
 'Shape_Area',
 'Shape_Leng',
 'TAX_ADD_L1',
 'TAX_ADD_L2',
 'TAX_ADD_L3',
 'TAX_CAPAC',
 'TAX_EXEMPT',
 'TAX_NAME',
 'TOTAL_TAX',
 'UNIT_INFO',
 'USE1_DESC',
 'USE2_DESC',
 'USE3_DESC',
 'USE4_DESC',
 'WSHD_DIST',
 'XUSE1_DESC',
 'XUSE2_DESC',
 'XUSE3_DESC',
 'XUSE4_DESC',
 'YEAR_BUILT',
 'ZIP',
 'ZIP4',
 'centroid_lat',
 'centroid_long',
 'Distance_Parcel_Lake_meters',
 'Yea

### Task 2. Feature Brainstorming

Our objective is to build a feature table with one row per lake-year, using grouped summary statistics. Here are effective strategies for feature construction:

1. **Numerical summaries:** Calculate group-level statistics (mean, median, standard deviation, IQR, etc.) for numeric variables.
2. **Categorical summaries:** For text data, consider:
   - **Success rates:** Compute proportions for binary variables (e.g., percent of homes with basements).
   - **Label cleaning:** Review and standardize unique labels to remove duplicates or inconsistencies.
   - **Broader categories:** Recode variables with many rare categories into a smaller, more meaningful set.
   - **Indicator columns:** Create indicator variables and aggregate them to show presence/absence or proportions (e.g., count of each property use type).

Review the variables you identified earlier and outline a feature construction strategy for each.

> <font color="orange"> **Feature Construction Strategy:**

**Numerical Variables - Use mean, median
- `EMV_LAND`, `EMV_BLDG`, `EMV_TOTAL`: Market value summaries (mean, median, std) - higher values may indicate more development
- `ACRES_POLY`, `ACRES_DEED`: Property size summaries - larger lots may have more runoff area
- `GARAGESQFT`: Average garage size - indicates impervious surface area
- `FIN_SQ_FT`: Average building footprint - more impervious surface
- `YEAR_BUILT`: Mean/median year built - track development timeline
- `NUM_UNITS`: Total units, mean units per property - population density indicator
- `TOTAL_TAX`, `TAX_CAPAC`: Tax summaries - economic indicators

**Binary/Categorical Variables - Use proportions/counts:**
- `BASEMENT`: Proportion with basements (Y vs N)
- `GARAGE`: Proportion with garages (Y vs N)
- `GREEN_ACRE`, `OPEN_SPACE`, `AG_PRESERV`: Count properties with these designations - less developed = potentially better water quality (Y/N)

**Aggregation Strategy:**
- Group by: `Monit_MAP_CODE1` (lake ID), `Year`, `distance_category` (within_500m, between_501_1600m, over_1600m)
- For each group, calculate the features above
- This gives us development characteristics at different distances from each lake over time
</font>

In [72]:
# Define variable categories for aggregation
numerical_vars = ['EMV_LAND', 'EMV_BLDG', 'EMV_TOTAL', 'ACRES_POLY', 'ACRES_DEED', 
                  'FIN_SQ_FT', 'YEAR_BUILT', 'NUM_UNITS', 'TOTAL_TAX', 'TAX_CAPAC']
categorical_vars = ['BASEMENT', 'GARAGE', 'GREEN_ACRE', 'OPEN_SPACE', 'AG_PRESERV']

In [73]:
# Aggregate parcel features by Monit_MAP_CODE1 (lake ID), Year, and distance_category
# Handling nulls by excluding them from aggregations, then filling remaining nulls with 0
(parcel_features_agg := 
    parcels_lazy
    .group_by(['Monit_MAP_CODE1', 'Year', 'distance_category'])
    .agg([
        # Count of parcels
        pl.len().alias('parcel_count'),
        
        # Keep centroid coordinates for joining (take first value since they should be the same for each lake)
        pl.col('centroid_lat').first().cast(pl.Utf8).alias('centroid_lat'),
        pl.col('centroid_long').first().cast(pl.Utf8).alias('centroid_long'),
        
        # Numerical variables - mean, median, std
        *[pl.col(var).cast(pl.Float64, strict=False).mean().alias(f'{var.lower()}_mean') 
          for var in numerical_vars],
        *[pl.col(var).cast(pl.Float64, strict=False).median().alias(f'{var.lower()}_median') 
          for var in numerical_vars if var not in ['NUM_UNITS', 'TAX_CAPAC']],
        *[pl.col(var).cast(pl.Float64, strict=False).std().alias(f'{var.lower()}_std') 
          for var in ['EMV_LAND', 'EMV_BLDG', 'EMV_TOTAL', 'ACRES_POLY']],
        
        # Special aggregations for NUM_UNITS
        pl.col('NUM_UNITS').cast(pl.Float64, strict=False).sum().alias('num_units_total'),
        
        # Categorical variables - proportions only
        *[((pl.col(var) == 'Y').sum() / pl.len()).alias(f'{var.lower()}_prop') 
          for var in categorical_vars],
    ])
    .collect()
    .fill_null(0)
)

Monit_MAP_CODE1,Year,distance_category,parcel_count,centroid_lat,centroid_long,emv_land_mean,emv_bldg_mean,emv_total_mean,acres_poly_mean,acres_deed_mean,fin_sq_ft_mean,year_built_mean,num_units_mean,total_tax_mean,tax_capac_mean,emv_land_median,emv_bldg_median,emv_total_median,acres_poly_median,acres_deed_median,fin_sq_ft_median,year_built_median,total_tax_median,emv_land_std,emv_bldg_std,emv_total_std,acres_poly_std,num_units_total,basement_prop,garage_prop,green_acre_prop,open_space_prop,ag_preserv_prop
str,str,str,u32,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""82012500-01""","""2005""","""within_500m""",60,"""45.10473""","""-92.88537""",200515.0,245978.333333,446493.333333,4.7275,8.033,0.0,0.0,0.0,2944.366667,0.0,225000.0,259700.0,503600.0,1.31,9.43,0.0,0.0,3130.0,90852.704082,140629.596533,214524.475208,6.028137,0.0,0.0,0.0,0.033333,0.0,0.0
"""82004200-02""","""2011""","""within_500m""",64,"""45.15143""","""-92.87435""",155312.5,102621.875,257934.375,9.32125,0.995,1074.78125,1108.625,0.5625,0.0,0.0,170000.0,111800.0,301200.0,6.03,0.0,1125.0,1930.0,0.0,101932.965056,105432.246308,172094.043337,10.614924,36.0,0.5625,0.53125,0.15625,0.0,0.03125
"""82005204-02""","""2005""","""between_501_1600m""",54,"""45.20838""","""-92.84404""",218922.222222,136331.481481,355253.703704,9.483519,17.82,0.0,0.0,0.0,1531.925926,0.0,177300.0,117900.0,321800.0,0.06,14.74,0.0,0.0,1566.0,155769.948499,137046.891609,223357.830949,14.383539,0.0,0.0,0.0,0.222222,0.0,0.0
"""82010300-01""","""2005""","""between_501_1600m""",1074,"""45.01535""","""-92.95542""",87510.986965,149052.141527,236563.128492,0.476518,0.90406,0.0,1649.845438,0.0,2247.748603,0.0,82500.0,141200.0,224500.0,0.0,0.25,0.0,1993.0,1912.0,175450.966618,160529.936297,243955.523806,4.204385,0.0,0.0,0.0,0.0,0.0,0.0
"""82009700-01""","""2007""","""between_501_1600m""",2781,"""44.87955""","""-92.96828""",96104.027328,206249.262855,302353.290183,0.0,1.519694,4.624236,1712.142035,1.0,12.30205,0.0,75000.0,207600.0,288600.0,0.0,0.25,0.0,1988.0,0.0,209427.244381,188183.557672,288105.316627,0.0,6.0,0.002157,0.0,0.007551,0.0,0.0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""82006800-01""","""2009""","""between_501_1600m""",26,"""45.2135""","""-92.81221""",158238.461538,72800.0,231038.461538,15.657692,15.657692,657.384615,763.923077,0.0,176238.461538,0.0,107000.0,0.0,202900.0,5.01,5.01,0.0,0.0,202900.0,172451.980741,97339.948634,196723.723179,21.33914,0.0,0.0,0.0,0.307692,0.0,0.0
"""10004402-01""","""2013""","""between_501_1600m""",523,"""44.85848""","""-93.67019""",76665.965583,159466.539197,236132.50478,0.977208,0.081033,2111.41109,1630.852772,0.0,3238.554493,0.0,82500.0,148100.0,226500.0,0.38,0.0,2424.0,2004.0,2886.0,47049.882516,164957.193939,196733.623744,3.434009,0.0,0.0,0.810707,0.0,0.0,0.0
"""19003000-01""","""2009""","""within_500m""",405,"""44.70907""","""-93.29404""",104612.592593,217665.925926,322278.518519,1.463333,0.0,2712.666667,1395.935802,0.755968,4945.187654,3869.540741,66500.0,191900.0,295500.0,0.51,0.0,2218.0,1974.0,3385.0,215571.433601,577723.264908,739709.01471,4.945331,285.0,0.0,0.0,0.0,0.0,0.0


In [74]:
# Check counts of distance_category for each year
parcel_features_agg.group_by(['Monit_MAP_CODE1','Year', 'distance_category']).agg(pl.len().alias('count')).sort(['Monit_MAP_CODE1', 'Year', 'distance_category'])
#need one year per year and map code

Monit_MAP_CODE1,Year,distance_category,count
str,str,str,u32
"""02000300-01""","""2004""","""between_501_1600m""",1
"""02000300-01""","""2004""","""over_1600m""",1
"""02000300-01""","""2004""","""within_500m""",1
"""02000300-01""","""2005""","""between_501_1600m""",1
"""02000300-01""","""2005""","""over_1600m""",1
…,…,…,…
"""99002487-01""","""2014""","""over_1600m""",1
"""99002487-01""","""2014""","""within_500m""",1
"""99002487-01""","""2015""","""between_501_1600m""",1


In [75]:
# Pivot to get one row per lake-year with distance_category as column prefixes
# Get all feature columns (excluding grouping/coordinate columns)
feature_cols = [col for col in parcel_features_agg.columns 
                if col not in ['Monit_MAP_CODE1', 'Year', 'distance_category', 'centroid_lat', 'centroid_long']]

# First, get centroid coordinates (take first value per lake since they vary slightly by distance_category)
centroids = parcel_features_agg.group_by(['Monit_MAP_CODE1', 'Year']).agg([
    pl.col('centroid_lat').first(),
    pl.col('centroid_long').first()
])

# Pivot without centroids in the index
(parcel_features_wide := parcel_features_agg
 .pivot(
     values=feature_cols,
     index=['Monit_MAP_CODE1', 'Year'],
     on='distance_category'
 )
 .join(centroids, on=['Monit_MAP_CODE1', 'Year'], how='left')
)

Monit_MAP_CODE1,Year,parcel_count_within_500m,parcel_count_between_501_1600m,parcel_count_over_1600m,emv_land_mean_within_500m,emv_land_mean_between_501_1600m,emv_land_mean_over_1600m,emv_bldg_mean_within_500m,emv_bldg_mean_between_501_1600m,emv_bldg_mean_over_1600m,emv_total_mean_within_500m,emv_total_mean_between_501_1600m,emv_total_mean_over_1600m,acres_poly_mean_within_500m,acres_poly_mean_between_501_1600m,acres_poly_mean_over_1600m,acres_deed_mean_within_500m,acres_deed_mean_between_501_1600m,acres_deed_mean_over_1600m,fin_sq_ft_mean_within_500m,fin_sq_ft_mean_between_501_1600m,fin_sq_ft_mean_over_1600m,year_built_mean_within_500m,year_built_mean_between_501_1600m,year_built_mean_over_1600m,num_units_mean_within_500m,num_units_mean_between_501_1600m,num_units_mean_over_1600m,total_tax_mean_within_500m,total_tax_mean_between_501_1600m,total_tax_mean_over_1600m,tax_capac_mean_within_500m,tax_capac_mean_between_501_1600m,tax_capac_mean_over_1600m,emv_land_median_within_500m,emv_land_median_between_501_1600m,…,year_built_median_between_501_1600m,year_built_median_over_1600m,total_tax_median_within_500m,total_tax_median_between_501_1600m,total_tax_median_over_1600m,emv_land_std_within_500m,emv_land_std_between_501_1600m,emv_land_std_over_1600m,emv_bldg_std_within_500m,emv_bldg_std_between_501_1600m,emv_bldg_std_over_1600m,emv_total_std_within_500m,emv_total_std_between_501_1600m,emv_total_std_over_1600m,acres_poly_std_within_500m,acres_poly_std_between_501_1600m,acres_poly_std_over_1600m,num_units_total_within_500m,num_units_total_between_501_1600m,num_units_total_over_1600m,basement_prop_within_500m,basement_prop_between_501_1600m,basement_prop_over_1600m,garage_prop_within_500m,garage_prop_between_501_1600m,garage_prop_over_1600m,green_acre_prop_within_500m,green_acre_prop_between_501_1600m,green_acre_prop_over_1600m,open_space_prop_within_500m,open_space_prop_between_501_1600m,open_space_prop_over_1600m,ag_preserv_prop_within_500m,ag_preserv_prop_between_501_1600m,ag_preserv_prop_over_1600m,centroid_lat,centroid_long
str,str,u32,u32,u32,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,str
"""82012500-01""","""2005""",60,27,null,200515.0,207470.37037,null,245978.333333,265766.666667,null,446493.333333,473237.037037,null,4.7275,7.49037,null,8.033,10.093704,null,0.0,0.0,null,0.0,0.0,null,0.0,0.0,null,2944.366667,3163.407407,null,0.0,0.0,null,225000.0,225000.0,…,0.0,null,3130.0,3448.0,null,90852.704082,83057.291695,null,140629.596533,149357.652839,null,214524.475208,215689.247883,null,6.028137,9.477534,null,0.0,0.0,null,0.0,0.0,null,0.0,0.0,null,0.033333,0.074074,null,0.0,0.0,null,0.0,0.0,null,"""45.10473""","""-92.88537"""
"""82004200-02""","""2011""",64,108,2,155312.5,169453.703704,133400.0,102621.875,158072.222222,616600.0,257934.375,327525.925926,750000.0,9.32125,14.922407,5.82,0.995,0.0,0.0,1074.78125,1632.333333,3605.0,1108.625,1577.944444,1999.0,0.5625,0.796296,1.0,0.0,0.0,0.0,0.0,0.0,0.0,170000.0,143800.0,…,1986.5,1999.0,0.0,0.0,0.0,101932.965056,114179.165183,0.0,105432.246308,127944.391274,0.0,172094.043337,173473.032928,0.0,10.614924,19.242134,0.0,36.0,86.0,2.0,0.5625,0.796296,1.0,0.53125,0.740741,1.0,0.15625,0.203704,0.0,0.0,0.0,0.0,0.03125,0.018519,0.0,"""45.15143""","""-92.87435"""
"""82005204-02""","""2005""",311,54,8,164426.366559,218922.222222,120175.0,109620.900322,136331.481481,129725.0,274047.266881,355253.703704,249900.0,1.221543,9.483519,8.7825,2.206688,17.82,17.5725,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1828.655949,1531.925926,1263.0,0.0,0.0,0.0,150000.0,177300.0,…,0.0,0.0,1896.0,1566.0,1073.0,138613.393476,155769.948499,96205.267899,147450.442349,137046.891609,156503.863485,268877.107047,223357.830949,251098.603285,3.775481,14.383539,15.290608,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.048

In [76]:
(final_dataset := wq.join(
    parcel_features_wide.with_columns(pl.col('Year').cast(pl.Int64)),
    left_on=['DNR_ID_Site_Number', 'Year'],
    right_on=['Monit_MAP_CODE1', 'Year'],
    how='left'
)).fill_null(0)

DNR_ID_Site_Number,LAKE_NAME,Year,latitude,longitude,avg_secchi_depth,avg_total_phosphorus,parcel_count_within_500m,parcel_count_between_501_1600m,parcel_count_over_1600m,emv_land_mean_within_500m,emv_land_mean_between_501_1600m,emv_land_mean_over_1600m,emv_bldg_mean_within_500m,emv_bldg_mean_between_501_1600m,emv_bldg_mean_over_1600m,emv_total_mean_within_500m,emv_total_mean_between_501_1600m,emv_total_mean_over_1600m,acres_poly_mean_within_500m,acres_poly_mean_between_501_1600m,acres_poly_mean_over_1600m,acres_deed_mean_within_500m,acres_deed_mean_between_501_1600m,acres_deed_mean_over_1600m,fin_sq_ft_mean_within_500m,fin_sq_ft_mean_between_501_1600m,fin_sq_ft_mean_over_1600m,year_built_mean_within_500m,year_built_mean_between_501_1600m,year_built_mean_over_1600m,num_units_mean_within_500m,num_units_mean_between_501_1600m,num_units_mean_over_1600m,total_tax_mean_within_500m,total_tax_mean_between_501_1600m,total_tax_mean_over_1600m,…,year_built_median_between_501_1600m,year_built_median_over_1600m,total_tax_median_within_500m,total_tax_median_between_501_1600m,total_tax_median_over_1600m,emv_land_std_within_500m,emv_land_std_between_501_1600m,emv_land_std_over_1600m,emv_bldg_std_within_500m,emv_bldg_std_between_501_1600m,emv_bldg_std_over_1600m,emv_total_std_within_500m,emv_total_std_between_501_1600m,emv_total_std_over_1600m,acres_poly_std_within_500m,acres_poly_std_between_501_1600m,acres_poly_std_over_1600m,num_units_total_within_500m,num_units_total_between_501_1600m,num_units_total_over_1600m,basement_prop_within_500m,basement_prop_between_501_1600m,basement_prop_over_1600m,garage_prop_within_500m,garage_prop_between_501_1600m,garage_prop_over_1600m,green_acre_prop_within_500m,green_acre_prop_between_501_1600m,green_acre_prop_over_1600m,open_space_prop_within_500m,open_space_prop_between_501_1600m,open_space_prop_over_1600m,ag_preserv_prop_within_500m,ag_preserv_prop_between_501_1600m,ag_preserv_prop_over_1600m,centroid_lat,centroid_long
str,str,i64,str,str,f64,f64,u32,u32,u32,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,str
"""82007700-01""","""Goggins Lake""",2006,"""45.13303508""","""-92.89283249""",0.956,0.102,69,137,9,204921.73913,171323.357664,151266.666667,147011.594203,199991.240876,230944.444444,351933.333333,371314.59854,382211.111111,5.32913,4.568102,3.212222,10.593043,8.489343,6.68,1234.115942,1503.124088,1555.666667,1369.971014,1502.131387,1526.777778,0.0,0.0,0.0,2050.202899,2182.788321,2745.111111,…,1981.0,1978.0,2366.0,2424.0,2884.0,128945.44089,102820.380406,87242.635792,121999.144809,193184.768868,275852.316064,195013.07925,228000.633558,316231.001186,9.497451,8.09445,4.598836,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.202899,0.116788,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"""45.12616""","""-92.8821"""
"""19002700-01""","""Crystal Lake""",2014,"""44.72296805""","""-93.27036573""",1.96,0.0244,1765,92,0,110654.787535,84493.478261,0.0,159819.320113,189930.434783,0.0,270474.107649,274423.913043,0.0,0.430295,0.606739,0.0,0.0,0.0,0.0,2228.563173,2573.119565,0.0,1821.904816,1872.815217,0.0,0.92068,0.923913,0.0,3317.18187,4132.423913,0.0,…,1979.0,0.0,2754.0,2792.5,0.0,138105.254957,120359.683298,0.0,82338.486478,129628.188275,0.0,168611.758605,230124.674938,0.0,0.769999,1.731464,0.0,1625.0,85.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.003399,0.0,0.0,0.0,0.0,0.0,"""44.71658""","""-93.26527"""
"""82012200-01""","""Pine Tree Lake""",2004,"""45.10231359""","""-92.95386928""",2.2625,0.027375,273,424,0,173076.556777,160262.971698,0.0,301198.168498,319631.839623,0.0,474274.725275,479894.811321,0.0,2.584652,2.91342,0.0,5.146777,6.124481,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3770.131868,3905.820755,0.0,…,0.0,0.0,3632.0,3458.0,0.0,111947.950582,135619.676884,0.0,332707.974039,260368.938186,0.0,398363.670937,309997.671648,0.0,7.7

In [77]:
# Write final dataset to CSV
final_dataset.write_csv('./data/final_dataset_with_parcel_features.csv')